# ML Model Example

This example provides a simply query tutorial for InfluxDB Cloud Dedicated. 

In [15]:
%pip install scikeras


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import influxdb_client_3 as InfluxDBClient3
import numpy as np
from keras.layers import Input, Dense, BatchNormalization, Normalization, LeakyReLU, Dropout
from keras.models import Model
from keras.regularizers import L1L2
from keras.callbacks import TensorBoard
from keras import Sequential

import plotly.io as pio
import plotly.express as px
from huggingface_hub import push_to_hub_keras

from keras.regularizers import L1L2
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

pio.renderers.default = "vscode"
import os, datetime


In [17]:
token = 'dzfx2CLrpmGjbTHmdbnw1cSB7Op89g6cQ2VD7EVRJYHiF8UN8bRqIaBK92TxnCa906qWjjV6KVk_TQrW0uH5IA=='
host = 'eu-central-1-1.aws.cloud2.influxdata.com'    
org = 'Jay-IOx'
db = 'robot'

client = InfluxDBClient3.InfluxDBClient3(token=token,
                         host=host,
                         org=org,
                         database=db) 
    
    


In [18]:
query = '''SELECT last("coord") as coord FROM "joints" WHERE time >= 1696587404700ms and time <= 1696587952888ms  GROUP BY time(2s) fill(linear)'''
table = client.query(query=query, language="influxql")
normal_data = table.to_pandas().drop(columns=['iox::measurement']).set_index('time').dropna(how='all')
fig = px.line(normal_data, y=["coord"], title='raw coord' )
fig.show()

In [19]:
query = '''SELECT last("coord") as coord FROM "joints" WHERE time >= 1696587936049ms and time <= 1696588323944ms  GROUP BY time(2s) fill(linear)'''
table = client.query(query=query, language="influxql")
anom_data = table.to_pandas().drop(columns=['iox::measurement']).set_index('time').dropna(how='all')
fig2 = px.line(anom_data, y=["coord"], title='raw coord')
fig2.show()

In [20]:
def get_model(l1_l2_tuple):
    def create_model():
        l1, l2 = l1_l2_tuple
        model = Sequential([
            Dense(8, activation='relu', activity_regularizer=L1L2(l1=l1, l2=l2)),
            BatchNormalization(),
            Dense(4, activation='relu'),
            Dense(4, activation='relu'),
            BatchNormalization(),
            Dense(1, activation='sigmoid')
        ])
        return model
    return create_model

param_grid = {'l1_l2_tuple': [(0.0, 0.0), (0.01, 0.0), (0.0, 0.01), (0.01, 0.01), (0.1, 0.0), (0.0, 0.1)]}

def get_estimator(l1_l2_tuple=(0.0, 0.0)):
    model = get_model(l1_l2_tuple)
    estimator = KerasRegressor(model=model, epochs=100, batch_size=32, verbose=0,
                               optimizer='adam', loss='mean_squared_error')
    return estimator

In [21]:
grid = GridSearchCV(estimator=get_estimator(), param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(normal_data, normal_data)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

ValueError: Invalid parameter l1_l2_tuple for estimator KerasRegressor.
This issue can likely be resolved by setting this parameter in the KerasRegressor constructor:
`KerasRegressor(l1_l2_tuple=(0.0, 0.0))`
Check the list of available parameters with `estimator.get_params().keys()`